<a href="https://colab.research.google.com/github/sava-stew/Principles-of-Data-Science-Assignment2/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Set up the dataframe

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/sava-stew/Principles-of-Data-Science-Assignment2/refs/heads/main/train.csv')

### **Remove the units from some of the attributes and only keep the numerical values (for example remove kmpl from "Mileage", CC from "Engine", bhp from "Power", and lakh from "New_Price").**

*While this is Part B of the assignment, it was done before Part A to assist with the task specified in Part A*

In [3]:
clean = df.copy()

In [4]:
def removeUnits(data):
  columns = ['Mileage', 'Engine', 'Power', 'New_Price']
  for column in columns:
    for row in range(len(data)):
      #check is value is a string
      if (isinstance(data[column][row], str)):
        x = data[column][row]
        #seperates the string on space: number and unit
        num = (x.split())[0]
        #replaces the initial value with the number only
        data.replace({column:x}, {column:float(num)}, inplace=True)
      else:
        continue

In [5]:
removeUnits(clean)
clean

<ipython-input-4-eb6815b017a6>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({column:x}, {column:float(num)}, inplace=True)


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13.00,1199.0,88.70,5.0,8.61,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461.0,63.10,5.0,NaN,3.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,6014,Maruti Swift VDI,Delhi,2014,27365,Diesel,Manual,First,28.40,1248.0,74.00,5.0,7.88,4.75
5843,6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,Manual,First,24.40,1120.0,71.00,5.0,NaN,4.00
5844,6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Manual,Second,14.00,2498.0,112.00,8.0,NaN,2.90
5845,6017,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,Manual,First,18.90,998.0,67.10,5.0,NaN,2.65


### **Look for the missing values in all the columns and either impute them (replace with mean, median, or mode) or drop them. Justify your action for this task.**

***Please note that all columns of the dataframe were considered, not just the columns for missing values. This is to prepare in the scenario that new rows were added to the original data set***

*Find the original dimensions of the dataframe to refer back to later*

In [6]:
clean.shape

(5847, 14)

*Identify the number of empty rows within the dataframe for the defined columns*

In [7]:
def emptyDrop(data):
  columns = ['Name', 'Fuel_Type', 'Engine', 'Seats']
  rows = len(df)
  empty = 0
  for row in range(rows):
    for column in columns:
      #checks if entry is empty
      if pd.isna(data.loc[row, column]):
        empty += 1
        break
  print(empty)

In [8]:
emptyDrop(clean)

38


*Name is the identifying column for each row. Without the Name column, there is no definitive way to identify the row. Because of this, it's dropped when empty.*

*Fuel_Type, Engine, and Seats are all dependent on the model of the car. Fuel_Type will be dependent on the engine, of which the size of is dependent on the mode. The number of seats is also dependent on the model fo the car. Because of this, these rows are dropped when the entries for these columns are empty, as finding the mean, mode, or median would misrepresent the car*

In [9]:
clean.dropna(subset=['Name', 'Fuel_Type', 'Engine', 'Seats'], inplace=True)
clean.shape

(5809, 14)

*Identify the number of empty rows within each column*

In [10]:
def findEmpty(data):
  columns = {
      'Location': None,
      'Year': None,
      'Kilometers_Driven': None,
      'Transmission': None,
      'Owner_Type': None,
      'Mileage': None,
      'Power': None,
      'New_Price': None,
      'Price': None
  }
  for column in columns:
    columns[column] = (data[column].isna()).sum()

  for column in columns:
    print(f'{column}: {columns[column]}')

In [11]:
findEmpty(clean)

Location: 0
Year: 0
Kilometers_Driven: 0
Transmission: 0
Owner_Type: 0
Mileage: 2
Power: 0
New_Price: 4994
Price: 0


In [12]:
def modeReplace(data, column):
  mode = data[column].mode()
  data.fillna({column:mode[0]}, inplace=True)

*Location, Transmission, and Owner_Type are dependent on the history of ownership of the car.*

*Most cars, at least within the provided date range, were offered in either manual and automatic transmission if purchased new. Because of this, empty cells could be substituded with the most frequently seem transmission type.*

*As these cars are for sale in the same country, and most likely within the same area, the Location and Owner_Type can be evaluated through mode as well to determine the most frequent, and most likely, values.*

In [13]:
modeReplace(clean, 'Location')
modeReplace(clean, 'Transmission')
modeReplace(clean, 'Owner_Type')

In [14]:
findEmpty(clean)

Location: 0
Year: 0
Kilometers_Driven: 0
Transmission: 0
Owner_Type: 0
Mileage: 2
Power: 0
New_Price: 4994
Price: 0


In [15]:
def meanReplace(data, column):
  mean = data[column].mean()
  mean = mean.round(2)
  data.fillna({column:mean}, inplace=True)

*Kilometers_Driven, Mileage, and Price are influenced by previous use and upkeep. Because of this, it can vary between cars but is not directly specific to the model of the car, meaning that the average/mean would be the best way to impute them.*

In [16]:
meanReplace(clean, 'Kilometers_Driven')
meanReplace(clean, 'Mileage')
meanReplace(clean, 'Price')

In [17]:
findEmpty(clean)

Location: 0
Year: 0
Kilometers_Driven: 0
Transmission: 0
Owner_Type: 0
Mileage: 0
Power: 0
New_Price: 4994
Price: 0


*Power is dependent on factors such as engine, but also the amount of upkeep that is done. To find impute values for Power, median was chosen to eliminate the extreme use cases (no upkeep vs. extreme upkeep).*

*New_Price has little to no values present previously in the column. Finding the mode and average would cause bias in the data, and too many rows were there to drop (would cause a great decrease in data available), so median was chosen.*

In [18]:
def medianReplace(data, column):
  median = data[column].median()
  data.fillna({column:median}, inplace=True)

In [19]:
medianReplace(clean, 'Power')
medianReplace(clean, 'New_Price')

In [20]:
findEmpty(clean)

Location: 0
Year: 0
Kilometers_Driven: 0
Transmission: 0
Owner_Type: 0
Mileage: 0
Power: 0
New_Price: 0
Price: 0


In [21]:
clean

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,11.48,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13.00,1199.0,88.70,5.0,8.61,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,11.48,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,11.48,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461.0,63.10,5.0,11.48,3.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,6014,Maruti Swift VDI,Delhi,2014,27365,Diesel,Manual,First,28.40,1248.0,74.00,5.0,7.88,4.75
5843,6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,Manual,First,24.40,1120.0,71.00,5.0,11.48,4.00
5844,6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Manual,Second,14.00,2498.0,112.00,8.0,11.48,2.90
5845,6017,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,Manual,First,18.90,998.0,67.10,5.0,11.48,2.65


### **Change the categorical variables ("Fuel_Type" and "Transmission") into numerical one hot encoded value**

In [22]:
encoded = pd.get_dummies(clean, columns=['Fuel_Type', 'Transmission'], dtype=int)
encoded

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,11.48,12.50,1,0,0,0,1
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,8.61,4.50,0,0,1,0,1
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,11.48,6.00,1,0,0,0,1
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,11.48,17.74,1,0,0,1,0
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,11.48,3.50,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,6014,Maruti Swift VDI,Delhi,2014,27365,First,28.40,1248.0,74.00,5.0,7.88,4.75,1,0,0,0,1
5843,6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,First,24.40,1120.0,71.00,5.0,11.48,4.00,1,0,0,0,1
5844,6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Second,14.00,2498.0,112.00,8.0,11.48,2.90,1,0,0,0,1
5845,6017,Maruti Wagon R VXI,Kolkata,2013,46000,First,18.90,998.0,67.10,5.0,11.48,2.65,0,0,1,0,1


### **Create one more feature and add this column to the dataset (you can use mutate function in R for this). For example, you can calculate the current age of the car by substracting "Year" value from the current year**

*Feature added was the age of the car, and was added to the end of columns*

In [23]:
encoded.loc[:, 'Age'] = 2025 - encoded['Year']
encoded

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Age
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,11.48,12.50,1,0,0,0,1,10
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,8.61,4.50,0,0,1,0,1,14
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,11.48,6.00,1,0,0,0,1,13
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,11.48,17.74,1,0,0,1,0,12
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,11.48,3.50,1,0,0,0,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,6014,Maruti Swift VDI,Delhi,2014,27365,First,28.40,1248.0,74.00,5.0,7.88,4.75,1,0,0,0,1,11
5843,6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,First,24.40,1120.0,71.00,5.0,11.48,4.00,1,0,0,0,1,10
5844,6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Second,14.00,2498.0,112.00,8.0,11.48,2.90,1,0,0,0,1,13
5845,6017,Maruti Wagon R VXI,Kolkata,2013,46000,First,18.90,998.0,67.10,5.0,11.48,2.65,0,0,1,0,1,12


### **Perform select, filter, rename, mutate, arrange, and summarize with group by operations (or their equivalent operations in Python) on this dataset.**

Filter (Python equivalent)

In [24]:
encoded.groupby('Age').filter(lambda x: all((x['Age'] > 0) & (x['Age'] < 10)))

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Age
5,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,11.36,2755.0,171.50,8.0,21.00,17.50,1,0,0,1,0,9
8,10,Maruti Ciaz Zeta,Kochi,2018,25692,First,21.56,1462.0,103.25,5.0,10.65,9.95,0,0,1,0,1,7
14,16,Honda Amaze S i-Dtech,Kochi,2016,58950,First,25.80,1498.0,98.60,5.0,11.48,5.40,1,0,0,0,1,9
15,17,Maruti Swift DDiS VDI,Jaipur,2017,25000,First,28.40,1248.0,74.00,5.0,11.48,5.99,1,0,0,0,1,8
26,28,Honda WRV i-VTEC VX,Kochi,2018,37430,First,17.50,1199.0,88.70,5.0,10.57,9.90,0,0,1,0,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5812,5982,Tata Hexa XTA,Jaipur,2016,39000,First,17.60,2179.0,153.86,7.0,21.00,13.50,1,0,0,1,0,9
5816,5987,Tata Tiago 1.2 Revotron XT,Kochi,2017,15386,First,23.84,1199.0,84.00,5.0,5.56,5.11,0,0,1,0,1,8
5825,5996,Jaguar XF 2.2 Litre Luxury,Kochi,2016,31150,First,16.36,2179.0,187.70,5.0,11.48,30.54,1,0,0,1,0,9
5827,5999,Tata Bolt Revotron XT,Chennai,2016,10000,First,17.57,1193.0,88.70,5.0,7.77,4.00,0,0,1,0,1,9


Select (Python equivalent)

In [25]:
encoded[['Price', 'New_Price']]

,Price,New_Price
0,12.50,11.48
1,4.50,8.61
2,6.00,11.48
3,17.74,11.48
4,3.50,11.48
...,...,...
5842,4.75,7.88
5843,4.00,11.48
5844,2.90,11.48
5845,2.65,11.48


Rename (Python equivalent)

In [26]:
encoded.rename(columns={'Name': 'Model'})

,Unnamed: 0,Model,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Age
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,11.48,12.50,1,0,0,0,1,10
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,8.61,4.50,0,0,1,0,1,14
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,11.48,6.00,1,0,0,0,1,13
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,11.48,17.74,1,0,0,1,0,12
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,11.48,3.50,1,0,0,0,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,6014,Maruti Swift VDI,Delhi,2014,27365,First,28.40,1248.0,74.00,5.0,7.88,4.75,1,0,0,0,1,11
5843,6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,First,24.40,1120.0,71.00,5.0,11.48,4.00,1,0,0,0,1,10
5844,6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Second,14.00,2498.0,112.00,8.0,11.48,2.90,1,0,0,0,1,13
5845,6017,Maruti Wagon R VXI,Kolkata,2013,46000,First,18.90,998.0,67.10,5.0,11.48,2.65,0,0,1,0,1,12


Mutate (Python equivalent)

In [27]:
encoded['Distance_Owner'] = encoded.groupby('Owner_Type')['Kilometers_Driven'].transform('mean')
encoded

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Age,Distance_Owner
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,11.48,12.50,1,0,0,0,1,10,55158.312239
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,8.61,4.50,0,0,1,0,1,14,55158.312239
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,11.48,6.00,1,0,0,0,1,13,55158.312239
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,11.48,17.74,1,0,0,1,0,12,71163.107338
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,11.48,3.50,1,0,0,0,1,12,55158.312239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5842,6014,Maruti Swift VDI,Delhi,2014,27365,First,28.40,1248.0,74.00,5.0,7.88,4.75,1,0,0,0,1,11,55158.312239
5843,6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,First,24.40,1120.0,71.00,5.0,11.48,4.00,1,0,0,0,1,10,55158.312239
5844,6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Second,14.00,2498.0,112.00,8.0,11.48,2.90,1,0,0,0,1,13,71163.107338
5845,6017,Maruti Wagon R VXI,Kolkata,2013,46000,First,18.90,998.0,67.10,5.0,11.48,2.65,0,0,1,0,1,12,55158.312239


Arrange (Python equivalent)

In [28]:
encoded.sort_values('Kilometers_Driven')

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Age,Distance_Owner
1315,1361,Maruti Alto 800 2016-2019 VXI,Mumbai,2019,171,First,24.70,796.0,47.30,5.0,11.48,3.60,0,0,1,0,1,6,55158.312239
5451,5606,Maruti Wagon R ZXI AMT 1.2,Mumbai,2019,600,First,21.50,1197.0,81.80,5.0,6.80,6.25,0,0,1,1,0,6,55158.312239
1123,1161,Tata Tigor 1.2 Revotron XTA,Ahmedabad,2018,1000,First,20.30,1199.0,84.00,5.0,11.48,6.85,0,0,1,1,0,7,55158.312239
5772,5941,Renault KWID RXL,Pune,2018,1000,First,25.17,799.0,53.30,5.0,4.45,3.40,0,0,1,0,1,7,55158.312239
1547,1598,Tata Xenon XT EX 4X2,Jaipur,2017,1000,First,13.49,2179.0,138.03,5.0,11.48,7.50,1,0,0,0,1,8,55158.312239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2997,3092,Honda City i VTEC SV,Kolkata,2015,480000,First,17.40,1497.0,117.30,5.0,11.48,5.00,0,0,1,0,1,10,55158.312239
344,358,Hyundai i10 Magna 1.2,Chennai,2009,620000,First,20.36,1197.0,78.90,5.0,11.48,2.70,0,0,1,0,1,16,55158.312239
1805,1860,Volkswagen Vento Diesel Highline,Chennai,2013,720000,First,20.54,1598.0,103.60,5.0,11.48,5.90,1,0,0,0,1,12,55158.312239
326,340,Skoda Octavia Ambition Plus 2.0 TDI AT,Kolkata,2013,775000,First,19.30,1968.0,141.00,5.0,11.48,7.50,1,0,0,1,0,12,55158.312239


Summarize (Python equivalent)

In [29]:
summarize = clean[['Fuel_Type', 'Price', 'New_Price']]
summarize.groupby('Fuel_Type').agg({'Price': ['mean'], 'New_Price': ['mean']})

,Price,New_Price
,mean,mean
Fuel_Type,,
Diesel,12.958496,13.687370
Electric,12.875000,12.530000
Petrol,5.773416,11.622746
